In [65]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Input, Lambda, RepeatVector, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
import os
from pretty_midi import PrettyMIDI

def parse_piano_roll(file_path, resolution=50):
    """
    Parse MIDI file to extract piano roll (binary or velocity-based).
    """
    midi = PrettyMIDI(file_path)
    piano_roll = midi.get_piano_roll(fs=resolution)  # Shape: (128, time_steps)
    return piano_roll.T  # Transpose to (time_steps, 128)

def prepare_piano_sequences(piano_roll, sequence_length=50):
    """
    Prepare sequences of piano roll data for LSTM.
    """
    sequences = []
    for i in range(len(piano_roll) - sequence_length):
        seq = piano_roll[i:i+sequence_length]
        sequences.append(seq)
    return np.array(sequences)

# Step 2: Build the LSTM-VAE model

latent_dim = 64
sequence_length = 50
input_dim = 128  # MIDI note range

# Encoder
inputs = Input(shape=(sequence_length, input_dim))
x = LSTM(128, return_sequences=True)(inputs)
x = LSTM(64)(x)
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

# Reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

# Decoder
decoder_inputs = Input(shape=(latent_dim,))
x = RepeatVector(sequence_length)(decoder_inputs)
x = LSTM(64, return_sequences=True)(x)
x = LSTM(128, return_sequences=True)(x)
outputs = TimeDistributed(Dense(input_dim, activation='sigmoid'))(x)

# Define models
encoder = Model(inputs, [z_mean, z_log_var, z], name="encoder")
decoder = Model(decoder_inputs, outputs, name="decoder")

# VAE model
vae_outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, vae_outputs, name="vae")

# Loss function

vae.compile(optimizer='adam')

# Step 3: Train the model

def train_model_piano_roll(midi_folder, epochs=10, batch_size=32):
    """
    Train the VAE on piano roll data from a folder.
    """
    all_sequences = []
    for file_name in os.listdir(midi_folder):
        file_path = os.path.join(midi_folder, file_name)
        if file_path.endswith('.mid'):
            try:
                piano_roll = parse_piano_roll(file_path)
                sequences = prepare_piano_sequences(piano_roll, sequence_length)
                
                if len(sequences) == 0:
                    print(f"No valid sequences generated from {file_name}. Skipping.")
                    continue
                
                all_sequences.extend(sequences)
            except Exception as e:
                print(f"Error processing {file_name}: {e}")
    
    if len(all_sequences) == 0:
        raise ValueError("No valid sequences found in the dataset. Check your MIDI files.")
    
    all_sequences = np.array(all_sequences)
    print("Dataset Shape:", all_sequences.shape)
    print("Contains NaNs:", np.isnan(all_sequences).any())
    print("Data Type:", all_sequences.dtype)
    print("Value Range: Min =", np.min(all_sequences), "Max =", np.max(all_sequences))
    
    vae.fit(all_sequences, epochs=epochs, batch_size=batch_size)

# Step 4: Generate music

def generate_music(latent_vector=None):
    """
    Generate music from the trained VAE model.
    """
    if latent_vector is None:
        latent_vector = np.random.normal(size=(1, latent_dim))
    generated_sequence = decoder.predict(latent_vector)
    return generated_sequence

# Example Usage
midi_folder = "/home/fariborz/courses/Research/data/maestro-v2.0.0/2006"  # Replace with your MIDI folder path
train_model_piano_roll(midi_folder, epochs=20)
latent_vector = np.random.normal(size=(1, latent_dim))
generated_sequence = generate_music(latent_vector)
print("Generated Music Sequence:", generated_sequence)


ValueError: No valid sequences found in the dataset. Check your MIDI files.

In [46]:
print(vae.summary())


Model: "vae"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_19 (InputLayer)     │ (None, 50, 3)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ [(None, 64), (None,    │       125,312 │
│                                 │ 64), (None, 64)]       │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 50, 3)          │       132,227 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 257,539 (1006.01 KB)

 Trainable params: 257,539 (1006.01 KB)

 Non-trainable params: 0 (0.00 B)

None


In [47]:
test_batch = np.random.rand(32, 50, 3)  # Example batch with matching shape
vae.predict(test_batch)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 548ms/step


array([[[-1.35277249e-02,  2.86675058e-04,  1.49522796e-02],
        [-3.05264071e-02,  4.79303906e-03,  3.35024446e-02],
        [-4.73046415e-02,  1.46503523e-02,  4.91696186e-02],
        ...,
        [-2.21401811e-01,  2.00594783e-01,  3.81022915e-02],
        [-2.21436411e-01,  2.00636327e-01,  3.81071568e-02],
        [-2.21466601e-01,  2.00673923e-01,  3.81120034e-02]],

       [[ 1.29920822e-02, -2.32630707e-02,  1.84037499e-02],
        [ 2.92061307e-02, -4.55401726e-02,  4.72427011e-02],
        [ 4.40736227e-02, -5.97609915e-02,  7.84114525e-02],
        ...,
        [ 7.10113943e-02,  6.94557326e-03,  1.67632148e-01],
        [ 7.10363388e-02,  6.93544699e-03,  1.67613715e-01],
        [ 7.10585788e-02,  6.92440104e-03,  1.67597339e-01]],

       [[ 1.20773269e-02, -6.28912961e-03, -3.43614654e-03],
        [ 2.42056623e-02, -1.07332570e-02, -1.77653634e-03],
        [ 3.13999094e-02, -1.09076956e-02,  6.67192088e-03],
        ...,
        [-4.27152403e-02,  1.02428466e-01,

In [48]:
print(inputs.shape, vae_outputs.shape)


(None, 50, 3) (None, 50, 3)


NameError: name 'all_sequences' is not defined